In [ ]:
import logging
import os
import pandas as pd
import json

from pathlib import Path

%load_ext autoreload
%autoreload 2

# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.FATAL))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

%aimport setup

%load_ext autoreload
%autoreload 2

from setup import *

# Track Layout
Calculate the layout of the dutch railway system


In [ ]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
layout = Layout(layout_file)

In [ ]:
edges_df = pd.DataFrame({"Outgoing routes": [len(n.outgoing) for n in layout.g_block.nodes.values() if len(n.outgoing) <= 25]})
hist = edges_df.hist(bins=25, )
plt.xlabel("Number of outgoing routes")
plt.ylabel("Number of occurrences")
plt.show()

# Experiment Runtime
Take a route of an agent with many stops, and run from start to every stop as an experiment

In [ ]:
scenario_file = "../data/prorail/scenarios/SHL-2025-06-30.json"
try:
    base_path = Path(__file__).parent
    file_path = (base_path / scenario_file).resolve()
    data = json.load(open(file_path))
except:
    data = json.load(open(scenario_file))
types = {x["name"]: x for x in data["types"]}
agents = []
for trainNumber, entry in enumerate(data["trains"]):
    trainNumber += 1
    move = entry["movements"][0]
    velocity = types[entry["trainUnitTypes"][0]]["speed"] / 3.6

    block_path = layout.get_path_for_agent(move, trainNumber, velocity)

    agent = Agent(trainNumber, move["startLocation"], move["endLocation"], velocity, move["startTime"],
                  endTime=move["endTime"],
                  startTimeHuman=str(timedelta(seconds=move["startTime"])),
                  endTimeHuman=str(timedelta(seconds=move["endTime"])),
                  blockPath=block_path,
                  trainNumber=entry["trainNumber"],
                  trainUnitTypes=entry["trainUnitTypes"],
                  stops=move["stops"]
    )
    agents.append(agent)

agent_df = pd.DataFrame([agent.__dict__ for agent in agents])
agent_df['blockPathLength'] = agent_df['blockPath'].map(len)

print(f"total blocks in path: {agent_df['blockPathLength'].sum()}")
print(f"total edges in graph: {edges_df['Outgoing routes'].sum()}")

agent_df

## Scenario

In [ ]:
scenario_file = "../data/prorail/scenarios/SHL-2025-06-30-TAD.json"
series_3500o = agent_df.loc[(agent_df['trainNumber'].str.startswith("35", na=False)) & (agent_df['trainNumber'].astype(int) % 2 == 1)]

# This train would be at 2700 at GV|6, we will be replanning its path till ASDZ|2
agent = series_3500o.iloc[1]
series_3500o

In [ ]:
# Replan these agents, -1 is planning in a new agent
agent_id = agent['id']
scenario = Scenario(layout, scenario_file, agent_id)

## Experiment
Define an experiment for every stop, and the original start and stop


In [ ]:
# Setup experiment
experiment_settings = [
    {
        "start_time": 2700.0,
        "origin": "GV|6",
        "destination": "ASDZ|2",
        "metadata": {
            "offset": 2,
        }
    },
    {
        "start_time": 2700.0,
        "origin": "GV|6",
        "destination": "ASDZ|2",
        "max_buffer_time": 900,
        "metadata": {
            "color": "Green",
            "label": "Buffer time",
            "offset": 1,
        }
    }, {
        "start_time": 2700.0,
        "origin": "GV|6",
        "destination": "ASDZ|2",
        "max_buffer_time": 900,
        "use_recovery_time": True,
        "metadata": {
            "color": "Blue",
            "label": "Recovery time",
        }
    }
]

experiments = setup_experiment(scenario, experiment_settings, default_direction=1)

### Blocking staircase diagram
Showing the route of the agent with the most stops, its quite long.


In [ ]:
for exp in experiments:
    exp.s.plot(agent_id, exp.buffer_times, exp.recovery_times, False)

In [ ]:
timeout = 600
run_experiments(experiments, timeout)

## Results


### ATF Plot

In [ ]:
pybooklogger.setLevel(logging.WARNING)
experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 5}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': -5}

kwargs = {"min_x": 2700, "max_x": 3100, "min_y": 4250, "max_y": 5200, "expected_arrival_time": 4920}
plot_experiments(experiments, **kwargs)

experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 0}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': 0}

plot_experiments([experiments[0]], **kwargs)
plot_experiments([experiments[1]], **kwargs)
plot_experiments([experiments[2]], **kwargs)

### Time statistics

In [ ]:
def sum_cols(df1, cols, name):
    df2 = df1.drop(columns=cols)
    df2[name] = df1[cols].sum(axis=1)
    return df2

time_df = pd.DataFrame([exp.get_running_time() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

setup_cols = ["track graph creation", "routing graph creation"]
recompute_cols = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks"]
search_cols = ["FlexSIPP search time"]

time_df = sum_cols(time_df, setup_cols, "Setup Time")
time_df = sum_cols(time_df, recompute_cols, "Recompute Time")
time_df = sum_cols(time_df, search_cols, "Search Time")
time_df

### Search Node Statistics


In [ ]:
nodes_df = pd.DataFrame([exp.get_complexity() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

nodes_df

### Output paths found


In [ ]:
for key, value in experiments[2].results[3].items():
    delayed_trains = {i: v for i,v in enumerate(value[0][4]) if float(v[0]) > 0}
    print(f"{key.replace('r-', '')}\nearliest departure: {int(min(float(value[0][1]), float(value[0][2])) / 60)}\ndepart before: {int(float(value[0][2]) / 60)}\narrive at: {int((float(value[0][1]) + float(value[0][3])) / 60)}\ndelays trains: {delayed_trains}")

### Path statistics


In [ ]:
for exp in experiments:
    print(f"Differend paths found for {exp.metadata['label']}: {sum(exp.results[2].values())}")